# Creating a Test System 

The data of three bus system from this website:

https://www.al-roomi.org/power-flow/3-bus-systems/system-iii

In [1]:
import pandapower as pp
import pandapower.plotting as plot
from numpy import pi
import os
import matplotlib.pyplot as plt
from pandas import DataFrame

Building the system, buses and lines  (now no transformers)

In [2]:
net = pp.create_empty_network(f_hz=60)

In [3]:
b1 = pp.create_bus(net, vn_kv=1, name="bus1", type="b")
b2 = pp.create_bus(net, vn_kv=1, name="bus2", type="b")
b3 = pp.create_bus(net, vn_kv=1, name="bus3", type="b")

### Insert the parameters of lines 
Note: In the table of lines data, there is a typo in the reactance of X13 it should be 0.18pu instead of 0.018pu

In [4]:
L12=pp.create_line_from_parameters(net, from_bus=b1, to_bus=b2, length_km=1, 
                               name="Line 1-2", r_ohm_per_km= 0.08, x_ohm_per_km=0.24, 
                               c_nf_per_km=1e-9, max_i_ka=1000)
L13=pp.create_line_from_parameters(net, from_bus=b1, to_bus=b3, length_km=1, 
                               name="Line 1-3", r_ohm_per_km= 0.02, x_ohm_per_km=0.06, 
                               c_nf_per_km=1e-9, max_i_ka=1000)
L23=pp.create_line_from_parameters(net, from_bus=b2, to_bus=b3, length_km=1, 
                               name="Line 2-3", r_ohm_per_km= 0.06, x_ohm_per_km=0.18, 
                               c_nf_per_km=1e-9, max_i_ka=1000) 

In [5]:
pp.create_ext_grid(net, bus=b1, vm_pu=1.05, va_degree=0, name="Grid Connection")
pp.create_gen(net, b2, p_mw=0.20, max_q_mvar=0.35, min_q_mvar=0, vm_pu=1.03, name="generator") 

0

In [6]:
pp.create_load(net, bus=b2, p_mw=0.50, q_mvar=0.20, name="Load")
pp.create_load(net, bus=b3, p_mw=0.60, q_mvar=0.25, name="Load")

1

In [7]:
net

This pandapower network includes the following parameter tables:
   - bus (3 elements)
   - load (2 elements)
   - gen (1 element)
   - ext_grid (1 element)
   - line (3 elements)

In [8]:
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,type,power_station_trafo
0,generator,1,0.2,1.03,NaN,0.0,0.35,1.0,False,True,None,NaN


In [9]:
net.bus

,name,vn_kv,type,zone,in_service
0,bus1,1.0,b,None,True
1,bus2,1.0,b,None,True
2,bus3,1.0,b,None,True


In [10]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Line 1-2,None,0,1,1.0,0.08,0.24,1.000000e-09,0.0,1000.0,1.0,1,None,True
1,Line 1-3,None,0,2,1.0,0.02,0.06,1.000000e-09,0.0,1000.0,1.0,1,None,True
2,Line 2-3,None,1,2,1.0,0.06,0.18,1.000000e-09,0.0,1000.0,1.0,1,None,True


In [41]:
# run power flow
# pp.runpp(net,enforce_q_lims=True)
pp.runpp(net,algorithm='gs', max_iteration=7, tolerance_mva=1e-03)

In [42]:
net.res_ext_grid

,p_mw,q_mvar
0,0.91311,0.240808


In [43]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.050000,0.000000,-0.91311,-0.240808
1,1.030000,-2.850369,0.30000,-0.050445
2,1.024756,-1.945315,0.60000,0.250000


In [44]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.229600,0.016542,-0.225755,-0.005007,0.003845,0.011535,0.126574,0.126574,0.126574,1.05,0.000000,1.030000,-2.850369,0.012657
1,0.683510,0.224266,-0.674123,-0.196104,0.009387,0.028162,0.395546,0.395546,0.395546,1.05,0.000000,1.024756,-1.945315,0.039555
2,-0.074139,0.055451,0.074624,-0.053997,0.000485,0.001454,0.051896,0.051896,0.051896,1.03,-2.850369,1.024756,-1.945315,0.005190


In [45]:
import numpy as np
from cmath import rect
mag=net.res_bus.vm_pu
ang=net.res_bus.va_degree

nprect = np.vectorize(rect)

c = nprect(mag, np.deg2rad(ang))
c

array([1.05      +0.j        , 1.02872569-0.05121964j,
       1.02416555-0.03478598j])

### Check out Ybus

In [46]:
Ybs=(round(net['_ppc']['internal']['Ybus'],3))
print(Ybs)

  (0, 0)	(6.25-18.75j)
  (0, 1)	(-1.25+3.75j)
  (0, 2)	(-5+15j)
  (1, 0)	(-1.25+3.75j)
  (1, 1)	(2.917-8.75j)
  (1, 2)	(-1.667+5j)
  (2, 0)	(-5+15j)
  (2, 1)	(-1.667+5j)
  (2, 2)	(6.667-20j)
